In [1]:
# !wget https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip
# !unzip dino-dragon.zip

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

2022-11-21 23:37:12.082739: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 23:37:12.172893: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-21 23:37:12.175406: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-21 23:37:12.175422: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

#### Data Preparation

The dataset contains around 1900 images of dinos and around 1900 images of dragons.

The dataset contains separate folders for training and test sets.
Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

    The shape for input should be (150, 150, 3)
    Next, create a convolutional layer (Conv2D):
        Use 32 filters
        Kernel size should be (3, 3) (that's the size of the filter)
        Use 'relu' as activation
    Reduce the size of the feature map with max pooling (MaxPooling2D)
        Set the pooling size to (2, 2)
    Turn the multi-dimensional result into vectors using a Flatten layer
    Next, add a Dense layer with 64 neurons and 'relu' activation
    Finally, create the Dense layer with 1 neuron - this will be the output
        The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use SGD with the following parameters:

    SGD(lr=0.002, momentum=0.8)

For clarification about kernel size and max pooling, check Office Hours.

In [3]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

2022-11-21 23:37:13.169492: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-21 23:37:13.169512: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-21 23:37:13.169541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (3511): /proc/driver/nvidia/version does not exist
2022-11-21 23:37:13.169719: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.compile(
    optimizer=SGD(learning_rate=0.002, momentum=0.8),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

#### Q1. Since we have a binary classification problem, what is the best loss function for us?

Note: since we specify an activation for the output layer, we don't need to set from_logits=True

#### binary crossentropy

#### Q2. What's the total number of parameters of the model? You can use the summary method for that.

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

#### 11215873

Generators and Training

For the next two questions, use the following data generator for both train and test sets:

ImageDataGenerator(rescale=1./255)

    We don't need to do any additional pre-processing for the images.
    When reading the data from train/val directories, check the class_mode parameter. Which value should it be for a binary classification problem?
    Use batch_size=20
    Use shuffle=True for both training and test sets.

For training use .fit() with the following params:

model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

In [6]:
image_gen = ImageDataGenerator(rescale=1./255)

train_gen = image_gen.flow_from_directory(
    directory='./train/',
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    class_mode='binary',
)

test_gen = image_gen.flow_from_directory(
    directory='./test/',
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    class_mode='binary',
)

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [7]:
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=test_gen
)

Epoch 1/10
80/80 [==============================] - 9s 109ms/step - loss: 0.6812 - accuracy: 0.6248 - val_loss: 0.6327 - val_accuracy: 0.5838
Epoch 2/10
80/80 [==============================] - 9s 111ms/step - loss: 0.5463 - accuracy: 0.7491 - val_loss: 0.6558 - val_accuracy: 0.5787
Epoch 3/10
80/80 [==============================] - 9s 111ms/step - loss: 0.4554 - accuracy: 0.8011 - val_loss: 0.4071 - val_accuracy: 0.8503
Epoch 4/10
80/80 [==============================] - 9s 114ms/step - loss: 0.4026 - accuracy: 0.8356 - val_loss: 0.3845 - val_accuracy: 0.8350
Epoch 5/10
80/80 [==============================] - 9s 114ms/step - loss: 0.3499 - accuracy: 0.8532 - val_loss: 0.3546 - val_accuracy: 0.8553
Epoch 6/10
80/80 [==============================] - 9s 117ms/step - loss: 0.3427 - accuracy: 0.8582 - val_loss: 0.3584 - val_accuracy: 0.8477
Epoch 7/10
80/80 [==============================] - 9s 117ms/step - loss: 0.2877 - accuracy: 0.8902 - val_loss: 0.5510 - val_accuracy: 0.7614
Epoch 

#### Q3. What is the median of training accuracy for all the epochs for this model?

In [8]:
round(np.median(history.history['accuracy']), 2)

0.86

#### 0.90

#### Q4. What is the standard deviation of training loss for all the epochs for this model?

In [9]:
round(np.std(history.history['loss']), 2)

0.14

#### 0.11

#### Data Augmentation

For the next two questions, we'll generate more data using data augmentations.

Add the following augmentations to your training data generator:

    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'

In [10]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = train_gen.flow_from_directory(
    directory='./train/',
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    class_mode='binary',
)

Found 1594 images belonging to 2 classes.


Let's train our model for 10 more epochs using the same code as previously. Make sure you don't re-create the model - we want to continue training the model we already started training.

In [11]:
history2 = model.fit(
    train_gen,
    epochs=10,
    validation_data=test_gen
)

Epoch 1/10
80/80 [==============================] - 14s 171ms/step - loss: 0.4928 - accuracy: 0.7654 - val_loss: 0.3431 - val_accuracy: 0.8503
Epoch 2/10
80/80 [==============================] - 14s 174ms/step - loss: 0.4126 - accuracy: 0.8187 - val_loss: 0.3210 - val_accuracy: 0.8731
Epoch 3/10
80/80 [==============================] - 14s 177ms/step - loss: 0.4201 - accuracy: 0.8024 - val_loss: 0.4033 - val_accuracy: 0.8147
Epoch 4/10
80/80 [==============================] - 14s 173ms/step - loss: 0.4106 - accuracy: 0.8162 - val_loss: 0.3441 - val_accuracy: 0.8376
Epoch 5/10
80/80 [==============================] - 14s 174ms/step - loss: 0.4047 - accuracy: 0.8130 - val_loss: 0.4860 - val_accuracy: 0.7640
Epoch 6/10
80/80 [==============================] - 14s 175ms/step - loss: 0.3793 - accuracy: 0.8262 - val_loss: 0.3917 - val_accuracy: 0.8249
Epoch 7/10
80/80 [==============================] - 14s 174ms/step - loss: 0.3841 - accuracy: 0.8325 - val_loss: 0.2909 - val_accuracy: 0.8706

#### Q5. What is the mean of test loss for all the epochs for the model trained with augmentations?

In [17]:
round(np.mean(history2.history['loss']), 2)

0.4

#### 0.37

#### Q6. What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

In [16]:
round(np.mean(history2.history['accuracy'][5:10]), 2)

0.84

#### 0.84